In [0]:
#jakas zmiana
import os
with open('tokens_and_api.txt', 'r') as file:
    exec(file.read())

#sas_token = tokens in tokens.txt
storage_account_name = "newadbprojektkakastorage"
container_name = "data"

spark.conf.set(
    f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SAS"
)
spark.conf.set(
    f"fs.azure.sas.token.provider.type.{storage_account_name}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider"
)
spark.conf.set(
    f"fs.azure.sas.fixed.token.{storage_account_name}.dfs.core.windows.net",
    sas_token
)

base_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"
print(f"Skonfigurowano dostęp do: {base_path}")

In [0]:
df_model = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{base_path}/final_results/df_weather_jfk.csv")

display(df_model.limit(5))

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

#usuwamy kolumny czasowe/identyfikatory, zostawiamy parametry lotu i pogody
feature_cols = ["DISTANCE", "HOURLYVISIBILITY", "HOURLYWindSpeed", "DAY_OF_WEEK"]

#linie lotnicze (tekst na liczby)
airline_indexer = StringIndexer(inputCol="AIRLINE", outputCol="AIRLINE_INDEX", handleInvalid="skip")

#składanie cech w jeden wektor
assembler = VectorAssembler(inputCols=feature_cols + ["AIRLINE_INDEX"], outputCol="raw_features", handleInvalid="skip")

#skalowanie 
scaler = StandardScaler(inputCol="raw_features", outputCol="features")

In [0]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

train_data, test_data = df_model.randomSplit([0.8, 0.2], seed=123)

rf_class = RandomForestClassifier(labelCol="CANCELLED", featuresCol="features", numTrees=50)

#pipeline dla klasyfikacji
pipeline_class = Pipeline(stages=[airline_indexer, assembler, scaler, rf_class])
model_cancel = pipeline_class.fit(train_data)

#predykcja
predictions_cancel = model_cancel.transform(test_data)

#AUC - im bliżej 1, tym lepiej
evaluator_auc = BinaryClassificationEvaluator(labelCol="CANCELLED", metricName="areaUnderROC")
auc = evaluator_auc.evaluate(predictions_cancel)

print(f"Skuteczność modelu (AUC) dla odwołań: {auc}")
display(predictions_cancel.select("CANCELLED", "prediction", "probability").limit(10))

In [0]:
display(predictions_cancel.filter((col("prediction") == 1) | (col("CANCELLED") == 1))
        .select("CANCELLED", "prediction", "probability"))

In [0]:
predictions_cancel.groupBy("CANCELLED", "prediction").count().show()

In [0]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

df_reg = df_model.filter(col("CANCELLED") == 0)
train_reg, test_reg = df_reg.randomSplit([0.8, 0.2], seed=123)

gbt_reg = GBTRegressor(labelCol="DEPARTURE_DELAY", featuresCol="features", maxIter=20)

#pipeline dla regresji
pipeline_reg = Pipeline(stages=[airline_indexer, assembler, scaler, gbt_reg])
model_delay = pipeline_reg.fit(train_reg)

#predykcja
predictions_delay = model_delay.transform(test_reg)

#RMSE
evaluator_rmse = RegressionEvaluator(labelCol="DEPARTURE_DELAY", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions_delay)

print(f"Średni błąd przewidywania (RMSE): {rmse} minut")
display(predictions_delay.select("DEPARTURE_DELAY", "prediction").limit(10))

# Section with df_weather_jfk_cleaned

In [0]:
df_model_2 = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{base_path}/final_results/df_weather_jfk.csv")

display(df_model_2.limit(5))

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from pyspark.sql.functions import to_timestamp, hour, month

# Filter and drop columns using Spark DataFrame syntax
df = df_model_2.filter(df_model_2['DEPARTURE_DELAY'] >= 0)
cols_to_drop = ['FLIGHT_NUMBER', 'CANCELLED', 'WEATHER_DELAY', 'DeparturedTimestamp','ORIGIN_AIRPORT','FLIGHT_NUMBER','AIRLINE','DISTANCE','DAY_OF_WEEK']
df = df.drop(*cols_to_drop)

# Convert to timestamp and extract hour and month
df = df.withColumn('ScheduledTimestamp', to_timestamp('ScheduledTimestamp'))
df = df.drop('ScheduledTimestamp')

# Convert to pandas DataFrame for sklearn
df_pd = df.toPandas()

# One-hot encoding
df_pd = pd.get_dummies(
    df_pd,
    drop_first=True
)

plt.figure(figsize=(10, 6))
corr = df_pd.corr()
sns.heatmap(corr[['DEPARTURE_DELAY']].sort_values(by='DEPARTURE_DELAY', ascending=False), annot=True, cmap='coolwarm')
plt.title('Korelacja cech z opóźnieniem (DEPARTURE_DELAY)')

plt.show() 

X = df_pd.drop('DEPARTURE_DELAY', axis=1)
y = df_pd['DEPARTURE_DELAY']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)
y_pred = model.predict(X)

feature_importances = pd.Series(model.feature_importances_, index=X.columns)

plt.figure(figsize=(10, 6))
feature_importances.nlargest(10).plot(kind='barh')
plt.title('Ważność cech w modelu (Feature Importance)')
plt.xlabel('Ważność')
plt.show() 

print("Dane po czyszczeniu (podgląd):")
print(df.head())

print("\nStatystyki modelu:")
print(f"MSE: {mean_squared_error(y, y_pred):.2f}")
print(f"R2: {r2_score(y, y_pred):.2f}")

print("\nPrzewidywania vs Rzeczywistość:")
comparison = pd.DataFrame({'Actual': y, 'Predicted': y_pred})

try:
    display(comparison.head(10))
except NameError:
    print(comparison)


In [0]:
display(df.limit(10))

In [0]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


df_pandas = df_model_2.toPandas()

# Zostawienie tylko delay >= 0
df_cleaned = df_pandas[df_pandas['DEPARTURE_DELAY'] >= 0].copy()

# Usuwanie zbędnych kolumn
columns_to_drop = ['FLIGHT_NUMBER', 'CANCELLED', 'WEATHER_DELAY', 'DeparturedTimestamp']

df_cleaned = df_cleaned.drop(columns=columns_to_drop, errors='ignore')

df_cleaned['ScheduledTimestamp'] = pd.to_datetime(df_cleaned['ScheduledTimestamp'])
df_cleaned['HOUR'] = df_cleaned['ScheduledTimestamp'].dt.hour
df_cleaned['MONTH'] = df_cleaned['ScheduledTimestamp'].dt.month
df_cleaned = df_cleaned.drop(columns=['ScheduledTimestamp'])

# Kodowanie zmiennych kategorycznych (One-Hot Encoding)
df_model = pd.get_dummies(df_cleaned, columns=['AIRLINE', 'ORIGIN_AIRPORT'], drop_first=True)

# ==========================================
# Wizualizacja 1: Korelacja
# ==========================================
plt.figure(figsize=(10, 6))
corr = df_model.corr()
sns.heatmap(corr[['DEPARTURE_DELAY']].sort_values(by='DEPARTURE_DELAY', ascending=False), annot=True, cmap='coolwarm')
plt.title('Korelacja cech z opóźnieniem (DEPARTURE_DELAY)')

plt.show() 

# Trenowanie modelu
X = df_model.drop('DEPARTURE_DELAY', axis=1)
y = df_model['DEPARTURE_DELAY']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)
y_pred = model.predict(X)

# Ewaluacja i Ważność cech
feature_importances = pd.Series(model.feature_importances_, index=X.columns)

# ==========================================
# Wizualizacja 2: Feature Importance
# ==========================================
plt.figure(figsize=(10, 6))
feature_importances.nlargest(10).plot(kind='barh')
plt.title('Ważność cech w modelu (Feature Importance)')
plt.xlabel('Ważność')
plt.show() 
print("Dane po czyszczeniu (podgląd):")
print(df_cleaned.head())

print("\nStatystyki modelu:")
print(f"MSE: {mean_squared_error(y, y_pred):.2f}")
print(f"R2: {r2_score(y, y_pred):.2f}")

print("\nPrzewidywania vs Rzeczywistość:")
comparison = pd.DataFrame({'Actual': y, 'Predicted': y_pred})

try:
    display(comparison.head(10))
except NameError:
    print(comparison)